In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
"""
This is the random model for the wizard of wikipedia task. There is no trainable model here only random retrieval of utterances of wizards.
This model runs on the test dataset.

Deepak Goyal <deepak.16je002137@ece.iitism.ac.in>
"""

In [0]:
import json
import os
import re
import math
import copy
import os
import string
import time

In [0]:
# Specify which machine you are running things
running_device='colab'

In [0]:
def raw_data_loader(running_device,name):
    """ 
        Function for loading json data into a list
    """
    
    file_dir=''
    file_name=name
    if running_device=='local':
        file_dir='/home/naive/Documents/rohit/Wizard Of Wikipedia/Dataset'
    elif running_device=='colab':
        file_dir='/content/drive/My Drive/Data/Wizard of Wikipedia/wizard_of_wikipedia'
    else:
        print("Invalid running device")
        return
    
    data=os.path.join(file_dir,file_name)
    
    json_data=None
    with open(data) as f:
        json_data=json.load(f)
    
    return json_data

In [14]:
json_data=None

print("Loading Raw Data into a list....")

t1=time.time()
json_data_seen=raw_data_loader(running_device,"test_random_split.json")
json_data_unseen=raw_data_loader(running_device,"test_topic_split.json")
t2=time.time()

print("Loading raw data took "+str(t2-t1)+" seconds")

Loading Raw Data into a list....
Loading raw data took 2.7358641624450684 seconds


In [18]:
print("Number of Conversations in seen test topics data: "+str(len(json_data_seen)))
print("Number of Conversations in unseen test topics data: "+str(len(json_data_unseen)))

Number of Conversations in seen test topics data: 965
Number of Conversations in unseen test topics data: 968


In [0]:
class vocabulary:
    """ Creates vocabulary for our corpus data of conversations"""
    
    def __init__(self):
        """
            self.word2Index(Dictionary): Maps words to tokens
            self.num_words(Scalar)     : Number of distinct words in the vocabulary
            
        """
        
        self.word2Index={'PAD':0,"<START>":1,"<END>":2}
        self.num_words=3
        
    def addSentence(self,sentence):
        """ 
            Adds sentences into vocabulary by splitting them
        """
        
        for word in sentence.split(" "):
            self.addWord(word)
    
    def addWord(self,word):
        
        if word not in self.word2Index.keys():
            self.word2Index[word]=self.num_words
            self.num_words+=1  

In [0]:
# Preprocessing functions for the data

def remove_punctuation(sentence):
    # thanks to https://stackoverflow.com/questions/265960/best-way-to-strip-punctuation-from-a-string
    return sentence.translate(str.maketrans('', '', string.punctuation)).lower().strip()

In [21]:
def load_data(data_json):
    
    context, knowledge, wizard=[],[],[]
    dict2cnt={}
    count=0
    
    N=len(data_json)
    for id in range(N):
        
        current_knowledge,current_wizard,context_knowledge,previous_context=[],[],[],[]
        tmp_context=[]
        
        current_conv=data_json[id]
        
        topic_chosen=remove_punctuation(current_conv['chosen_topic'])
        tmp_context.append(topic_chosen)
        
        conversation_length=len(current_conv['dialog'])
        
        for i in range(conversation_length):
            
            if current_conv['dialog'][i]['speaker']=='0_Wizard':
                
                dict2cnt[count]=len(current_conv['dialog'][i]['retrieved_passages'])
                count=count+1
                
                for x in range(len(current_conv['dialog'][i]['retrieved_passages'])):
                    
                    idx=current_conv['dialog'][i]['retrieved_passages'][x]
                    
                    for value in idx.values():
                        
                        for j in range(len(value)):
                            value[j]=remove_punctuation(value[j])
                            
                        current_knowledge.append(value)
                
                wizard_dialog=remove_punctuation(current_conv['dialog'][i]['text'])
                
                current_wizard.append(wizard_dialog)
                tmp_context.append(wizard_dialog)
            
            else:
                apperentice_dialog=remove_punctuation(current_conv['dialog'][i]['text'])
                tmp_context.append(apperentice_dialog)
                
        knowledge.append(current_knowledge)
        wizard.append(current_wizard)
        
        final_context=[]
        flag=True
        
        if current_conv['dialog'][0]['speaker']=='0_Wizard':
            flag=False
        
        for i in range(len(tmp_context)):
            temp=[]
            
            for j in range(i+1):
                temp.append(tmp_context[j])
                
            if flag:
                if i%2!=0 and len(final_context)<len(current_wizard):
                    final_context.append(temp)
                    
            else:
                if i%2==0 and len(final_context)<len(current_wizard):
                    final_context.append(temp)
        
        context.append(final_context)
    
    return context, knowledge, wizard, dict2cnt

print("Getting test conversation's context, knowledge, wizard utterances..")
t1=time.time()
context, knowledge, wizard, dict2cnt=load_data(json_data_seen)        
us_context, us_knowledge, us_wizard, us_dict2cnt=load_data(json_data_unseen)
t2=time.time()
print("This took: "+str(t2-t1)+" seconds")

Getting test conversation's context, knowledge, wizard utterances..
This took: 1.1849308013916016 seconds
